In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import sqlite3
import re

# 1. Парсер

In [2]:
driver = webdriver.Chrome()

In [11]:
conn = sqlite3.connect('panorama_news.db') # создаю базу данных, где будут храниться тексты и их метаиформация
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS news 
(id INTEGER PRIMARY KEY AUTOINCREMENT, topic, link, title, date, text)
""")
conn.commit()

def db_write(id_text, topic, link, title, date, text): # функция, которая записывает нужную информацию в базу данных
    cur.execute(
        """
        INSERT INTO news 
            (id, topic, link, title, date, text) 
            VALUES (?, ?, ?, ?, ?, ?)
        """, (
            id_text, topic, link, title, date, text)
    )
    conn.commit()

In [4]:
id_text = 0
parsed = []
def parse_review(review, topic): #парсинг отдельного текста
    global id_text
    url = 'https://panorama.pub' + review.attrs['href']  #ссылка на текст
    req = driver.get(url)
    page = driver.page_source
    if len(page) > 0:
        soup = BeautifulSoup(page, 'html.parser')
        title = soup.find('h1', {'class': 'font-bold text-2xl md:text-3xl lg:text-4xl pl-1 pr-2 self-center'}).text.strip('\n')
        if title not in parsed:
            metainfo = soup.find('div', {'class': 'flex flex-col gap-x-3 gap-y-1.5 flex-wrap sm:flex-row'}).text
            if 'сегодня' in metainfo:
                date = '16 окт. 2022 г.'
            elif 'вчера' in metainfo:
                date = '15 окт. 2022 г.'
            elif 'позавчера' in metainfo:
                date = '14 окт. 2022 г.'
            else:
                date = re.findall(r'[1-3]?\d \w{2,4}\.? \d{4} г.', metainfo)[0]
            text = soup.find('div', {'class': 'entry-contents pr-0 md:pr-8'}).text.replace('\n', ' ') # текст
            db_write(id_text, topic, url, title, date, text) # записываю в базу данных
            parsed.append(title)
            id_text += 1

In [5]:
def parse_page(link, topic): #парсинг страницы с текстами
    time.sleep(2)
    req = driver.get(link)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('a', {'class': 'flex flex-col rounded-md hover:text-secondary hover:bg-accent/[.1] mb-2'}) #нахожу отдельные тексты на странице
    for review in reviews:
        time.sleep(2)
        parse_review(review, topic) #отправляю текст на парсинг

In [13]:
for i in range(2,47):
    url = f'https://panorama.pub/science?page={i}'
    parse_page(url, 'Наука')

# 2. Анализ

In [1]:
import json
from pymorphy3 import MorphAnalyzer
import sqlite3
import re

In [2]:
m = MorphAnalyzer() # анализирую с помощью пайморфи

In [14]:
from nltk.tokenize import casual_tokenize
from nltk.tokenize import sent_tokenize

In [6]:
conn = sqlite3.connect('panorama_news.db')
cur = conn.cursor()

In [49]:
texts = []
id_text = 0
for text in cur.execute('SELECT * FROM news'): # из базы данных прохожусь по каждому тексту
    sentences = []
    word_count = 0
    for sent in sent_tokenize(text[5]): # делю каждый текст на предложения
        words = []
        for word in casual_tokenize(sent, preserve_case=False): # каждое предложение делю на слова
            if word.isalpha(): # слова, написанные буквами анализирую
                pos = []
                lemma = []
                for analysis in m.parse(word): # добавляю все возможные леммы и пос-тэги
                    pos.append(analysis.tag.POS)
                    lemma.append(analysis.normal_form)
                dct_word = {'word': word, 'POS': list(set(pos)), 'lemma': list(set(lemma))} 
                # создаю словарь анализа слова с самим словом, всеми возможными его пос-тэгами и леммами
                words.append(dct_word) # добавляю словарь в список слов текущего предложения
                word_count += 1 
        dct_sent = {'sentence': sent, 'words': words}
        # создаю словарь предложения с самим предложением и списком всех слов в нём
        sentences.append(dct_sent) # добавляю его в список предложений текущего текста
    if word_count >= 100: 
        # если в тексте насчиталось больше 100 словоформ, создаю его словарь с метаинформацией и добавляю в общий список текстов
        dct_text = {'id': id_text, 'link': text[2], 'date': text[4], 'title': text[3], 'sentences': sentences}
        id_text += 1
        texts.append(dct_text)
    if len(texts) >= 100: # как только набралось 100 текстов, останавливаюсь
        break

In [50]:
with open('panorama_corpus.json', 'w', encoding='utf-8') as f: # записываю корпус в json
    json.dump(texts, f, ensure_ascii=False, indent='\t')

Ниже представлена структура корпуса

In [ ]:
[
    {'id': id_text,
     'link': link,
     'date': date,
     'title': title,
     'sentences':
         [
             {'sentence': text,
              'words':
                  [
                      {'word': token,
                       'POS':
                           ['POS_1', 'POS_2', ...]
                        'lemma':
                           ['lemma_1', 'lemma_2', ...]
                      },
                      # word_2, word_3, ...
                  ]
             },
             # sentence_2, sentence_3, ...
         ]
    },
    # text_2, text_3, ...
]